In [ ]:
from notebook_prelude import *

In [ ]:
dataset_helper.get_all_available_dataset_names()
#[filename_utils.get_dataset_from_filename(x) for x in dataset_helper.get_all_cached_graph_datasets(graph_type=TYPE_CONCEPT_MAP)]

In [ ]:
g = nx.read_gml('tmp/concept-graph.graph')
fig, ax = plt.subplots()
nx.draw_networkx(g, ax=ax)
ax.grid(False)

